In [1]:
#load dragonn tutorial utilities 
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

## Ingesting data into tileDB 

In [2]:
from seqdataloader.dbingest import * 

The header of the input task file should contain (one or more) of the following fields: 
    * dataset (this one's required -- it's a unique label for your dataset) 
    * pval_bigwig 
    * fc_bigwig 
    * count_bigwig 
    * idr_peak
    * overlap_peak 
    * ambig_peak 
    
The file paths can be either local or web-based URL's. 

In [3]:
!cat tasks.dbingest.tsv

dataset	idr_peak	fc_bigwig	ambig_peak
ENCFF209DJG	https://www.encodeproject.org/files/ENCFF209DJG/@@download/ENCFF209DJG.bed.gz	https://www.encodeproject.org/files/ENCFF842XRQ/@@download/ENCFF842XRQ.bigWig	hg38.blacklist.bed.gz


You can run the ingest code as a python function: 

In [ ]:
args={"tiledb_metadata":"tasks.dbingest.tsv",
      "tiledb_group":"hepg2_dnase_encode",
     "overwrite":True,
     "chrom_sizes":"hg38.chrom21.sizes",
     "chrom_threads":1,
     "task_threads":1}

ingest(args)

Or you can run the code as a script: 

In [ ]:
!db_ingest --tiledb_metadata tiledb_metadata_adpd.txt \
       --tiledb_group /mnt/lab_data2/annashch/alzheimers_parkinsons/adpd_tiledb \
       --overwrite \
       --chrom_sizes hg38.chrom.sizes \
       --chrom_threads 1 \
       --task_threads 1

In [ ]:
#we can examine the array 
import tiledb 
data=tiledb.DenseArray("hepg2_dnase_encode/ENCFF209DJG.chr21",'r')
subset=data[30000000:31000000]
print(subset.keys())

In [ ]:
subset['fc_bigwig'][0:1000]

In [ ]:
subset['idr_peak'][0:1000]

## Genomewide classification labels 

In [5]:
from seqdataloader.labelgen import *
classification_params={
    'task_list':"tasks.labelgen.tsv",
    'outf':"classificationlabels.SummitWithin200bpCenter.tsv.gz",
    'output_type':'gzip',
    'chrom_sizes':'hg38.chrom.sizes',
    'chroms_to_keep':['chr21'],
    "store_positives_only":True,
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'task_threads':10,
    'chrom_threads':4,
    'allow_ambiguous':True,
    'labeling_approach':'peak_summit_in_bin_classification'
    }
genomewide_labels(classification_params)



Using TensorFlow backend.


creating dictionary of bed files and bigwig files for each task:
ENCFF209DJG
ENCFF605WXD
ENCFF073ORT
ENCFF618VMC
creating chromosome thread pool
launching thread pool
pre-allocated df for chrom:chr21with dimensions:(934180, 7)
got peak subset for chrom:chr21 for task:ENCFF618VMC
finished chromosome:chr21 for task:ENCFF618VMC
got peak subset for chrom:chr21 for task:ENCFF605WXD
got peak subset for chrom:chr21 for task:ENCFF073ORT
finished chromosome:chr21 for task:ENCFF605WXD
finished chromosome:chr21 for task:ENCFF073ORT
got peak subset for chrom:chr21 for task:ENCFF209DJG
finished chromosome:chr21 for task:ENCFF209DJG
writing output chromosomes:chr21
done!


## Genomewide regression labels 

In [6]:
regression_params={
    'task_list':"tasks.labelgen.tsv",
    'outf':"regressionlabels.all_genome_bins_regression.hdf5",
    'output_type':'hdf5',
    'chrom_sizes':'hg38.chrom.sizes',
    'store_values_above_thresh': 0,
    'chroms_to_keep':['chr21'],
    'bin_stride':50,
    'left_flank':400,
    'right_flank':400,
    'bin_size':200,
    'threads':10,
    'subthreads':4,
    'labeling_approach':'all_genome_bins_regression'
    }
genomewide_labels(regression_params)


creating dictionary of bed files and bigwig files for each task:
ENCFF209DJG
ENCFF605WXD
ENCFF073ORT
ENCFF618VMC
creating chromosome thread pool
launching thread pool
pre-allocated df for chrom:chr21with dimensions:(934180, 7)
starting chromosome:chr21 for task:ENCFF209DJG
finished chromosome:chr21 for task:ENCFF209DJG
starting chromosome:chr21 for task:ENCFF605WXD
finished chromosome:chr21 for task:ENCFF605WXD
starting chromosome:chr21 for task:ENCFF073ORT
finished chromosome:chr21 for task:ENCFF073ORT
starting chromosome:chr21 for task:ENCFF618VMC
finished chromosome:chr21 for task:ENCFF618VMC
writing output chromosomes:chr21
done!


let's examine the output dataframe for the regression case: 

In [9]:
regression_data=pd.read_hdf("regressionlabels.all_genome_bins_regression.hdf5")

In [10]:
regression_data.head()

ENCFF209DJG  ENCFF605WXD  ENCFF073ORT  ENCFF618VMC
CHR   START   END                                                        
chr21 5030250 5031250          0.0          0.0     0.003884          0.0
      5030300 5031300          0.0          0.0     0.008200          0.0
      5030350 5031350          0.0          0.0     0.012516          0.0
      5030400 5031400          0.0          0.0     0.013811          0.0
      5030450 5031450          0.0          0.0     0.009927          0.0

In [18]:
regression_negatives=pd.read_hdf("universal_negatives.regressionlabels.all_genome_bins_regression.hdf5")
regression_negatives.head

<bound method NDFrame.head of           CHR     START       END
0       chr21         0      1000
1       chr21        50      1050
2       chr21       100      1100
3       chr21       150      1150
4       chr21       200      1200
5       chr21       250      1250
6       chr21       300      1300
7       chr21       350      1350
8       chr21       400      1400
9       chr21       450      1450
10      chr21       500      1500
11      chr21       550      1550
12      chr21       600      1600
13      chr21       650      1650
14      chr21       700      1700
15      chr21       750      1750
16      chr21       800      1800
17      chr21       850      1850
18      chr21       900      1900
19      chr21       950      1950
20      chr21      1000      2000
21      chr21      1050      2050
22      chr21      1100      2100
23      chr21      1150      2150
24      chr21      1200      2200
25      chr21      1250      2250
26      chr21      1300      2300
27      chr21     

for the classification case, we specified "store_positives_only", so the script generated two dataframes: 
    * Universal negatives 
    * Dataframe where each bin is >0 for at least one task 

In [14]:
classification_pos=pd.read_csv("classificationlabels.SummitWithin200bpCenter.tsv.gz",sep='\t',header=0)

In [15]:
classification_pos.head()

,CHR,START,END,ENCFF209DJG,ENCFF605WXD,ENCFF073ORT,ENCFF618VMC
0,chr21,5065150,5066150,0.0,0.0,1.0,0.0
1,chr21,5065200,5066200,0.0,0.0,1.0,0.0
2,chr21,5065250,5066250,0.0,0.0,1.0,0.0
3,chr21,5065300,5066300,0.0,0.0,1.0,0.0
4,chr21,5101350,5102350,1.0,0.0,1.0,0.0


In [16]:
classification_neg=pd.read_csv("universal_negatives.classificationlabels.SummitWithin200bpCenter.tsv.gz",sep='\t',header=0)

In [17]:
classification_neg.head()

,CHR,START,END
0,chr21,0,1000
1,chr21,50,1050
2,chr21,100,1100
3,chr21,150,1150
4,chr21,200,1200
